# Notes from example
- telling names for functions
- use type hinting for arguments and function returns
- 1 feature per function

# Produktiv-Code

In [ ]:
import cv2
import numpy as np
from accelerate.test_utils.testing import get_backend
from transformers import pipeline, Pipeline
from PIL import Image

In [ ]:
# Video Handling
#TODO: Prüfe Nachbearbeitung Bild erforderlich
#TODO: Data-Import like in Example

def open_video(path: str) -> cv2.VideoCapture:
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()
    return cap

def next_image_from_video(cap: cv2.VideoCapture) -> Optional[Image.Image]:
    ret, frame = cap.read()
    if not ret:
        return None

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame_rgb)
    return image

def image_to_video(image: str, video_writer: cv2.VideoWriter):
    #GGF noch postprocessing des videos, z.B. Normalisieren
    #TODO Übergabe Bildtyp anpassen und cv2 Umwandlung korrekt auswählen
    # Achtung: VideoWriter erwartet 3 Kanäle, wenn isColor=True – also ggf. Graustufenbild nach BGR konvertieren
    depth_bgr = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Frame ins Video schreiben
    video_writer.write(depth_bgr)

def apply_model(frame: Image.Image ,model_selection: str) -> Image.Image:
    device, _, _ = get_backend()
    if model_selection == "Depth-Anything-V2-base-hf":
        checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
        pipe = pipeline("depth-estimation", model=checkpoint, device=device)
        predictions = pipe(frame)
        image_w_pred = predictions['depth']

    # If all models are called in the same way via the transformers library/pipeline, we can remove the general part and only put the parameterization in the if clause

    return image_w_pred

def get_videowriter(cap: cv2.VideoCapture, target_path: str) -> cv2.VideoWriter:
    # Video Schreiber erstellen
    # Videoeigenschaften holen
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # VideoWriter vorbereiten (MP4 mit H.264)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Alternativ: 'avc1', 'XVID', 'H264'
    video_writer = cv2.VideoWriter(target_path, fourcc, fps, (width, height), isColor=False)

def convert_video(path: str):
    # Video durchlaufen, Fehlermeldungen berücksichtigen
    cap = open_video(path)

    target_path = "" #TODO Funktion zur Definition des Targetpath 
    #Video Ausgabe initialisieren
    video_writer = get_videowriter(cap,target_path)

    #TODO Schleife zum Video-Durchlauf
    #TODO sicheres Abfangen von Fehlern beim Video öffnen
    next_image_from_video(cap)
    new_image = apply_model()
    image_to_video(new_image)
    
    cap.release()
    video_writer.release() #Nach schließen wird das Video abgespeichert





# Test Snippets